In [ ]:
import datetime
import math
import pandas as pd
from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))
import matplotlib.pyplot as plt
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import beta
from scipy import stats
from scipy.stats import chi2, chi2_contingency
import random
import os

for more information pls check gitHub reporsitory: https://github.com/caglanakpinar/a_b_test

In [ ]:
date_start = '2019-08-05' # this the day that my sample ab_tesitng data sset is started.
is_from_db = False # Normally, this notebook runs from my local postgre sql db. 
# However, I also attached the .csv file of each data frame and zip them

# In order to run sample data sets, make sure unzip ab.test_data.csv
# This data is randomly created. It is not related to any companies or web sites. For more infırmation about how randomly it creates, you may check a_b_test_data_creator.ipnb

### Problem Defination
- I want to make a improvement on my web page. I wonder that how this development effect my clients attributes.
#### Main Question; is the any difference between each screen of attractions?
- In order to answer this question I will sample from 

### Data I need?
- I need binary transaction for each client of visits. It must be well splited into Control and Validation Sets.
- Control and Validation sets of sample size can be equal if it is possible.
- In orer to sample, for each variation I use RAndom Sampling. 
- However, I segmented clients list, in each cluster of my client data set, randomly I picked my data sets.
- In order to find the right sample size for each segment. This calculation will worked well;
### segment_sample_size_control =  (segment_client_count/ total_client_count) * total_control_sample_size
- Segmentation is another problem, but I have already segmented my clients by RFM segmentation.
- I am going to calculate for whole sample of Control and Validation Sets, also, I will calculate my CTR values for each RFM segmets too.
- After I have sampled my data, I will calculate the Click Through Rate for each metrics which are login, basket, login_screen, ordered for Control and Validation samples.
- CTR_login = login_count / session_count
- CTR_basket = basket_count / session_count
- CTR_basket = order_screen_count / session_count
- CTR_basket = order_count / session_count



# Data Access

- The data set has clients of transactions.
- Sample data set has two variation of a web site. It also has the clients of attributes after they see two variation.
#### Metrics of check are; 
##### login: 
- It shows client sign in after seeing the screen in their visits. (0: Not Signing in (Not click) , 1: Signing in (click))
##### basket: 
- It shows adding any product to thier basket in visits after they sign in (0: Didn`t add anything in basket, 1: There is a basket (click)) 
##### order_screen: 
- It shows being at payment screen after they add any products to thier baskets in their visits. (0: didn`t visit payment screen, 1: visited payment screen (click))
##### order_count:
- It shows ordering action of each transaction of client after their visits. (0: not ordered, 1: ordered (click))

In [ ]:
if is_from_db:
    query = """
            SELECT
                client_id,
                is_control,
                sum(CASE WHEN session IS NOT NULL THEN 1 ELSE 0 END) as session_count,
                sum(CASE WHEN login IS NOT NULL THEN 1 ELSE 0 END) as login_count,
                sum(CASE WHEN basket IS NOT NULL THEN 1 ELSE 0 END) as basket_count,
                sum(CASE WHEN order_screen IS NOT NULL THEN 1 ELSE 0 END) as order_screen_count,
                sum(CASE WHEN ordered IS NOT NULL THEN 1 ELSE 0 END) as order_count
            
            FROM designingtest WHERE date = '{}'
            GROUP BY client_id, is_control
    """.format(str(date_start)[0:10])
    ab_test = pd.read_sql(query, connection_abtestdb)    
else:
    ab_test = pd.read_csv("ab_test_start_date.csv")  
ab_test.head(1)

In [ ]:
if is_from_db:
    query = """
            SELECT frequency_segment, monetary_segment, recency_segment, client_id
            FROM client_segments_daily WHERE date = '{}'
    """.format(str(datetime.datetime.strptime(date_start, '%Y-%m-%d') + datetime.timedelta(days=2))[0:10])
    segments = pd.read_sql(query, connection_clientsdb)
else:
    segments = pd.read_csv("segments.csv") 
display(segments.head(1))

In [ ]:
ab_test = pd.merge(ab_test, segments, on='client_id', how='left')
ab_test['rfm'] = ab_test.apply(lambda row: 
                               str(row['recency_segment']) + '_' + 
                               str(row['frequency_segment']) + '_' + str(row['monetary_segment']) ,axis=1)
ab_test.head()

In [ ]:
print('list of RFM segments :')
list(ab_test['rfm'].unique())

# A / B Test - Frequentist Approach
## Two Sample t- test:
- t Test allows us to answer the main question 'is the any difference between each screens of attractions?'
- Two sample t-test of confidence interval will show us how accuratelly new variation of screen worked. Two sample are Control VS Validation
- I will decided to implement my test with significance level of % 95.
- By accepting significance level of % 95, we also assume that %5 of error by H0 hyphothesis.
- In t - test we will work, We assume that CTR values will have t-distribution with estimated parameters X_mean and Standart Deviations of each control and Validation Sample.
- CTR ~ t(X_mean, Variance)

### Estimated Parameters 
- In roder to calculate t value for t-test, we need estimated parameters. 
- Beacuse we don`t know the population we are working on the sample of it which we assume that it represents it.
#### In this problem CTR values are paremters for t- distributions.


#### Let`s calcualte parameters (CTR values fo each metrics for each segments);

In [ ]:
# I will calculate CTR values for each segment and client.
# Later, in order to calculate main data sets of CTR values, calculating the avreage of them will help me to find the CTR_main.
ab_test_pivot = ab_test.pivot_table(index=['rfm', 'is_control'], aggfunc={'session_count':'sum',
                                                                    'login_count':'sum',
                                                                   'basket_count':'sum',
                                                                   'order_screen_count':'sum',
                                                                   'order_count':'sum'}).reset_index()
# CTR_login = login_ratio
ab_test_pivot['login_ratio'] = ab_test_pivot['login_count'] / ab_test_pivot['session_count']
# CTR_basket = basket_ratio
ab_test_pivot['basket_ratio'] = ab_test_pivot['basket_count'] / ab_test_pivot['session_count']
# CTR_order_screen = order_screen_ratio
ab_test_pivot['order_screen_ratio'] = ab_test_pivot['order_screen_count'] / ab_test_pivot['session_count']
# CTR_order = order_ratio
ab_test_pivot['order_ratio'] = ab_test_pivot['order_count'] / ab_test_pivot['session_count']
ab_test_pivot = ab_test_pivot.reset_index()
ab_test_pivot.head()

### Let`s see how Control and Validation set of CTR values are distributedon each metrics

In [ ]:
for i in ['login_ratio', 'basket_ratio', 'order_screen_ratio', 'order_ratio']:
    print(i, " Check:")
    x1 = ab_test_pivot[ab_test_pivot['is_control'] == 0][i].values
    y1 = ab_test_pivot[ab_test_pivot['is_control'] == 0]['index'].values
    x2 = ab_test_pivot[ab_test_pivot['is_control'] == 1][i].values
    y2 = ab_test_pivot[ab_test_pivot['is_control'] == 1]['index'].values
    print('CTR (Parameters for t - distribution) Preditor of Control and Validation Group :')
    print("CTR_Control_" + i + " :", round(np.mean(x1), 2), " || ", "CTR_Validation_" + i + " :", round(np.mean(x2), 2))
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    
    ax1.scatter(x1, y1, s=10, c='b', marker="s", label='control')
    ax1.scatter(x2,y2, s=10, c='r', marker="o", label='validation')
    plt.legend(loc='upper left');
    plt.show()

# Let`s calculate the t - tests of each metrics

### H0: There is no difference on control and validation test sets.
### H1: Thre statistically difference between Control And Validation sets of click ratios

In [ ]:
# This is calculation of t value for t- distribution
def calculate_t_test(p1, p2, click1, click2, n1, n2):
    p = (click1 + click2) / (n1 + n2)
    t = (p1 - p2) / sqrt(( p * (1 - p)) / ((1 / n1) + (1 / n2)) )
    df = n1 + n2 - 2
    pval = 1 - stats.t.sf(np.abs(t), df)*2  # two-sided pvalue = Prob(abs(t)>tt)
    confidence_limit = 1.96 * sqrt(((p1 * (1 - p1)) / n1) + ((p2 * (1 - p2)) / n2))
    confidence_intervals = [abs(p1 - p2) - confidence_limit, abs(p1 - p2) + confidence_limit]
    return pval, confidence_intervals

In [ ]:
metrics = ['login_count', 'basket_count', 'order_screen_count', 'order_count']
n1 = sum(list(ab_test_pivot[ab_test_pivot['is_control'] == 0]['session_count']))
n2 = sum(list(ab_test_pivot[ab_test_pivot['is_control'] == 1]['session_count']))

for metric in metrics:
    print("----- ", i, " Check :", "-----")
    print("H0 : There is no difference on control and validation test sets on ", metric, " ratios.")
    print("p_control_", metric, " = ", "p_validation_", metric)
    print("H1: Thre statistically difference between Control And Validation sets ", metric, " ratios.")
    print("p_control_", metric, " != ", "p_validation_", metric)
    print()
    click1 =  sum(list(ab_test_pivot[ab_test_pivot['is_control'] == 1][metric]))
    click2 =  sum(list(ab_test_pivot[ab_test_pivot['is_control'] == 0][metric]))
    print("Click Control :",  click1, " || \n Click Validation :",  click2, 
          " || \n Sample Size Control :", n1, " || \n Sample Size Validation :", n2)
    print()
    pval, confidence_intervals = calculate_t_test(click1 / n1, click2 / n2, click1, click2, n1, n2)
    print("p - value :", round(1- pval, 2), " || \n",
          "Control_CTR_" + metric + " :", round(click1 / n1, 2), " || \n",
          "Validation_CTR_" + metric + " :", round(click2 / n2, 2), " || \n",
          "-- HO REJECTED! --" if 1 - pval > 0.975 or 1 - pval < 0.25 else "-- HO ACCEPTED! --")
    print("left - right tail :", confidence_intervals)
    print()
    
    

### Result:
- Assumption of H0 is rejeceted for each metrics. There is significant difference between Control and Validation CTR statistically.

### T - Test Result Of Confidence;
- T test is a parametric test that needs estimeted paramters of data. In this problem our paramter is CTR which represents p ratios.
- However, t - test gives us a very general results of similarities between Control and Validation CTR`s, It is better to test the diffrence and similarities between Control and VAlidation set without using CTR parameter.
- In this case, It is  better to use one of the non-parametric test which is Chi_Square.

# Let`s see with Chi-Squared Test"

- t test is a test which runs with parameter of p proportion. 
- However, most of the time we don`t know the exact estimated p proportion.
- That is why, it is better to test also with Non Parametrics Test.
- Proportion of distributions are Chi-Squared. 
- This test is also one tail test. It is also convient to our data set.

In [ ]:
# In order to valvulate Chi-Squared value for Chi-Square distribution,
# I need click and non click count of control and Validation sets.
def chi_square_test(total_control, click_control, total_validation, click_validation):
    observed_df = pd.DataFrame([
                    {'test': 'control', 'click': click_control, 'non_click': total_control - click_control},
                    {'test': 'validation', 'click': click_validation, 'non_click': total_control - click_validation},
                    {'test': 'total', 'click': click_control + click_validation, 
                     'non_click': (total_control - click_control) + (total_control - click_validation)},
                  ])
    observed_df['total'] = observed_df['click'] + observed_df['non_click']
    
    total = list(observed_df[observed_df['test'] == 'total']['total'])[0]
    
    x2_val = 0
    for g in ['control', 'validation']:
        for v in ['click', 'non_click']:
            _expected = (list(observed_df[observed_df['test'] == g]['total'])[0] * 
                         list(observed_df[observed_df['test'] == 'total'][v])[0]) / total
            _observed = list(observed_df[observed_df['test'] == g][v])[0]
            
            #print(_expected, _observed, pow((_observed - _expected), 2) / _expected)
            x2_val += pow((_observed - _expected), 2) / _expected
    pval = chi2.cdf(x=x2_val, df=1)
    print(pval, "HO REJECTED!" if pval > 0.95 else "HO ACCEPTED!")
    return [pval, "HO REJECTED!" if pval > 0.95 else "HO ACCEPTED!"]

# let`s run the each metric of Chi - Squared Test

In [ ]:
metrics = ['login_count', 'basket_count', 'order_screen_count', 'order_count']
n1 = sum(list(ab_test[ab_test['is_control'] == 0]['session_count']))
n2 = sum(list(ab_test[ab_test['is_control'] == 1]['session_count']))

for metric in metrics:
    print(metric, " Check:")
    print("H0 : There is no difference on control and validation test sets on ", metric, " ratios.")
    print("p_control_", metric, " = ", "p_validation_", metric)
    print("H1: Thre statistically difference between Control And Validation sets ", metric, " ratios.")
    print("p_control_", metric, " != ", "p_validation_", metric)
    click1 =  sum(list(ab_test[ab_test['is_control'] == 0][metric]))
    click2 =  sum(list(ab_test[ab_test['is_control'] == 1][metric]))
    
    print(click1, click2, n1, n2)
    outputs = chi_square_test(n1, click1, n2, click2)
    

### Result of Chi - Square:
- Same as t- test, it is obvious that sample Control is significantly different from sample Validation set.

# Frequentist Approach disadvantes;
- However we test are differences between two sample, we are still not sure about the estimater parameter value. 
- In t - test we assume that CTR values of each samples represents population CTR values well enough to test We also assume that if make mistake about that assumption it will be less than %5.
- CTR value which is parameter of t - distribution, is an unknown parameter of population Mean statistic. It is estimator. We don`t know exact mean. We only use estimated of it.
- On the other hand we try Chi-Squared non-parametric test in order to get ride of estimated parameter value.
- In this case we also test the difference of two sample, without knowledge of CTR values. 

# Bayesian Approach

- Let`s check for each metrics with Bayesian Process. 
### Why we do this?
- It is because each ratios for metric is of parameters not well known. We have the data, but we don`t know the parameters.
- Bayesian Process allows us to finding the ratios of distribution without using the parameter.
- It deals with clicks data as P(X|Q) distribution X ~ Bernolli Because it only has (0, 1) which are click or non-click. (x : transaction of each metric ex: login_count_ x = [1,1,1, 0, 1, ... , 1,0,0])
- The knowledge of the parameter Q probability of P(Q) which is a probability distribution. Values of X are between [0, 1] That is why it will have ßeta Distibution with parameter of a and b.
- P(X|Q); X ~ Bernolli and P(Q); ßeta(a, b) are priors. 
- Bayes Theorem:  P(Q|X) = P(X|Q) * P(Q). This means when we have ratio of knowledge of parameter(P(Q)) and probability of X data given P(X|Q), I can calculate  parameter with this priors.
- Bernolli (X ~ Bernolli) * Beta(Q ~ ßeta) Distributions also shown Beta(a - 1 + ∑X, b - 1 + ∑X) which is posterior.
- So, if we check beta pdf of values by using a = "click count" and b = " non click count", at the end we might get the parameter without assuming as it`s prediction as like we did at t-test.

In [ ]:
print("Estimated CTR for login count :" ,
      round(np.mean(list(ab_test_pivot[ab_test_pivot['is_control'] == 0]['login_ratio'])),2),
      round(np.mean(list(ab_test_pivot[ab_test_pivot['is_control'] == 1]['login_ratio'])),2))
p_value_login_control = list(ab_test[ab_test['is_control'] == 0]['login_count'])
p_value_login_validation = list(ab_test[ab_test['is_control'] == 1]['login_count'])

In [ ]:
metrics = ['login_count', 'basket_count', 'order_screen_count', 'order_count']
control = ab_test[ab_test['is_control'] == 0]
validation = ab_test[ab_test['is_control'] == 1]

def plot_p_value(x, y1, y2, p1, p2):
    plt.plot(x, y1, label="real p: %.4f" % p1)
    plt.plot(x, y2, label="real p: %.4f" % p2)
    plt.legend()
    plt.show()

# check each metrics 
for metric in metrics:
    print(metric, " Check:")
    print("Is Validation Set of ", metric, "Conversion Rate is bigger than Control set?")

    x = np.linspace(0, 1, 200) # this is for ploting the results
    a_control, b_control = 1, 1 # at first we started we can assign beta a, b shape parameters defult 1 and 1.
    a_val, b_val = 1, 1 # it must be two beta distribution that is why I assing for each validaiton and control set of a and b.
    c_val, v_val = list(control[metric]), list(validation[metric])
    for ind in list(range(max(len(control), len(validation)))):
        # control set a, b updating
        # for each click update a and b paramters. a = click, b = non click
        try:
            a_control += c_val[ind] # click
            b_control += abs(c_val[ind] - 1) # non-click
        except:
            if ind + 1 == len(control):
                print("out of index")
        
        # validation set a, b updating
        try:
            a_val += v_val[ind] # click
            b_val += abs(v_val[ind] - 1) # non-click
        except:
            if ind + 1 == len(validation):
                print("out of index")
                
    
        if ind in [10, 50, 100, 1000]: # check for given indexes to make sure it is updating CTR parameter.
            print("number of sample :", ind)
            y_control = beta.pdf(x, a_control, b_control)  # beta probabilty density function for control a, b parameter
            y_validation = beta.pdf(x, a_val, b_val) # beta probabilty density function for  validation a, b parameter
            plot_p_value(x, y_control, y_validation, a_control / (a_control + b_control), a_val / (a_val + b_val))
            
    # check how many times vaidation CTR wins, control CTR
    control_p_values = stats.beta.rvs(a_control, b_control, size=len(control))
    validation_p_values = stats.beta.rvs(a_val, b_val, size=len(control))      
    wins = validation_p_values > control_p_values
    # ratio of wins: It also represents how accuratelly Validation Set has bigger CTR than Control Set.  
    print("percent of times that Validation wins : ", np.mean(wins) * 100) 


    
                       

# Result of Bayesian Approach:
- It is better with unkown parameter at the end it finalize with estimated parameter with Beta Distribution.


# Let`s run whole A/B Test for each days and check the results

In [ ]:
# This function allows me to calculate the sample sizes, clieck count of each metrics
def get_metrics(df, metric, day, rfm):
    print("H0 : There is no difference on control and validation test sets on ", metric, " ratios.")
    print("p_control_", metric, " = ", "p_validation_", metric)
    print("H1: Thre statistically difference between Control And Validation sets ", metric, " ratios.")
    print("p_control_", metric, " != ", "p_validation_", metric)
    print(day)
    df_1 = df[df['day'] <= day]
    df_1 = df_1 if rfm is None else df_1[df_1['rfm'] == rfm]
    click1 =  sum(list(df_1[df_1['is_control'] == 0][metric]))
    click2 =  sum(list(df_1[df_1['is_control'] == 1][metric]))
    n1 = len(df_1[df_1['is_control'] == 0])
    n2 = len(df_1[df_1['is_control'] == 1])
    return click1, click2, n1, n2, list(df_1[df_1['is_control'] == 0][metric]), list(df_1[df_1['is_control'] == 1][metric])

In [ ]:
# this function calculates t value, and confidence intervl of each metric
def definition_of_t_test(click1, click2, n1, n2):   
    pval, confidence_intervals = calculate_t_test(click1 / n1, click2 / n2, click1, click2, n1, n2)
    print(1 - pval, click1 / n1, click2 / n2, "HO REJECTED!" if 1 - pval > 0.975 or 1 - pval < 0.25 else "HO ACCEPTED!")
    return [1 - pval, "HO REJECTED!" if 1 - pval > 0.975 or 1 - pval < 0.25 else "HO ACCEPTED!", confidence_intervals]

In [ ]:
# bayesian method of function to calculate probability of X data given of parameter
def bayesian_approach(c_val, v_val):
    
    def plot_p_value(x, y1, y2, p1, p2):
        plt.plot(x, y1, label="control p: %.4f" % p1)
        plt.plot(x, y2, label="validation p: %.4f" % p2)
        plt.legend()
        plt.show()
    

    x = np.linspace(0, 1, 200)
    a_control, b_control = 1, 1
    a_val, b_val = 1, 1
    number_of_sample = max(len(c_val), len(v_val))
    for ind in list(range(max(len(c_val), len(v_val)))):
        # control set a, b updating
        try:
            a_control += c_val[ind] # click
            b_control += abs(c_val[ind] - 1) # non-click
        except:
            if ind + 1 == len(c_val):
                print("out of index")
        
        # validation set a, b updating
        try:
            a_val += v_val[ind] # click
            b_val += abs(v_val[ind] - 1) # non-click
        except:
            if ind + 1 == len(v_val):
                print("out of index")
    
        if ind in [100, int((number_of_sample *3) / 4)]:
            print("number of sample :", ind)
            y_control = beta.pdf(x, a_control, b_control)
            y_validation = beta.pdf(x, a_val, b_val)
            plot_p_value(x, y_control, y_validation, a_control / (a_control + b_control), a_val / (a_val + b_val))
            
    control_p_values = stats.beta.rvs(a_control, b_control, size=len(c_val))
    validation_p_values = stats.beta.rvs(a_val, b_val, size=len(v_val))  
    sample_size = min(len(control_p_values), len(validation_p_values))
    wins = validation_p_values[:sample_size] > control_p_values[:sample_size]
    return  np.mean(wins)

In [ ]:
# this functions to merge all outputs into one arrary to finalized
def test_control_validation_sets(ab_test_total, metric, day, rfm):
    click_control, click_vald, n_control, n_vald, _control, _validation = get_metrics(ab_test_total, metric, day, rfm)
    print(click_control, click_vald, n_control, n_vald, day, metric)
    t_test_outputs = definition_of_t_test(click_control, click_vald, n_control, n_vald)
    print("chi squared check :")
    chi_square_test_outputs = chi_square_test(n_control, click_control, n_vald, click_vald)
    print("Bayesian approach :")
    wins = bayesian_approach(_control, _validation)
    print(str(round(wins, 3) * 100), " % times validation set of CTR is bigger than control set of CTR")
    return click_control / n_control, click_vald / n_vald, np.mean(wins) ,t_test_outputs, chi_square_test_outputs

In [ ]:
metrics = ['login_count', 'basket_count', 'order_screen_count', 'order_count']

In [ ]:
# make sure you are running with csv file.
if is_from_db:
    days = list(pd.read_sql("""SELECT date FROM designingtest GROUP BY date ORDER BY date ASC""", connection_abtestdb)['date'])
    query = """
            SELECT
                client_id,
                is_control,
                date as day,
                sum(CASE WHEN session IS NOT NULL THEN 1 ELSE 0 END) as session_count,
                sum(CASE WHEN login IS NOT NULL THEN 1 ELSE 0 END) as login_count,
                sum(CASE WHEN basket IS NOT NULL THEN 1 ELSE 0 END) as basket_count,
                sum(CASE WHEN order_screen IS NOT NULL THEN 1 ELSE 0 END) as order_screen_count,
                sum(CASE WHEN ordered IS NOT NULL THEN 1 ELSE 0 END) as order_count
            
            FROM designingtest WHERE date >= '{}' and date <= '{}'
            GROUP BY client_id, is_control, date
    """.format(str(min(days))[0:10], str(max(days))[0:10])
    ab_test_total = pd.read_sql(query, connection_abtestdb)
    ab_test_total = pd.merge(ab_test_total, segments, on='client_id', how='left')
    ab_test_total['rfm'] = ab_test_total.apply(lambda row: 
                                               str(row['recency_segment']) + '_' + 
                                               str(row['frequency_segment']) + '_' + str(row['monetary_segment']) ,axis=1)
    
    print(len(ab_test_total))
    rfm_segments = list(ab_test_total['rfm'].unique())
    ab_test_total.head(1)
else:
    try:
        ab_test_total = pd.read_csv('ab_test_data.csv')
        ab_test_total['day'] = ab_test_total['day'].apply(lambda x: datetime.datetime.strptime(str(x)[0:10], '%Y-%m-%d'))
        days = list(ab_test_total['day'].unique())
        rfm_segments = list(ab_test_total['rfm'].unique())
    except:
        print("please check file has been unziped.")
        print("please check ab_test_data.csv is at same directory with .ipynb notebook.")

# it will take for a while, approximatelly  7 mins

In [ ]:
df_list = df_list_rfm = []
p_value_contol = p_value_validation = win_list = []
for day in days:
    print(" day :", str(day)[0:10])
    for metric in metrics:        
        p_control, p_validation, win_ratio, t_test, chi_square = test_control_validation_sets(ab_test_total, metric, day, None)
        df_list.append({'p_control': p_control, 
                        'p_validation': p_validation, 
                        'win_ratio': win_ratio, 
                        't_test_p_value': t_test[0],
                        't_test_H0': t_test[1],
                        't_test_left_tail': t_test[2][0],
                        't_test_right_tail': t_test[2][1],
                        'chi_square_p_value': chi_square[0],
                        'chi_square_H0': chi_square[1],
                        'bayesian_approach_confidence': win_ratio,
                        'day': day,
                        'metrics': metric
                       })
        # check for each segment of Control and Validation Samples
        for rfm in rfm_segments:
            print("rfm segment :", rfm)
            try: # days there isn`t any transaction of some segments of clients
                p_control, p_validation, win_ratio, t_test, chi_square = test_control_validation_sets(ab_test_total, metric, day, 
                                                                                                       rfm)
                
                df_list_rfm.append({'p_control': p_control, 
                                    'p_validation': p_validation, 
                                    'win_ratio': win_ratio, 
                                    't_test_p_value': t_test[0],
                                    't_test_H0': t_test[1],
                                    't_test_left_tail': t_test[2][0],
                                    't_test_right_tail': t_test[2][1],
                                    'chi_square_p_value': chi_square[0],
                                    'chi_square_H0': chi_square[1],
                                    'bayesian_approach_confidence': win_ratio,
                                    'rfm': rfm,
                                    'day': day,
                                    'metrics': metric
                                   })
            except:
                print("not enough sample to test!")
            

In [ ]:
daily_ab_test_results = pd.DataFrame(df_list)
daily_ab_test_results_rfm = pd.DataFrame(df_list_rfm)

In [ ]:
daily_ab_test_results.to_csv('daily_ab_test_results.csv')
daily_ab_test_results_rfm.to_csv('daily_ab_test_results_rfm.csv')

# RESULTS

## Login CTR
- Variation Screen has an impact to clients of logins almost %80. O average It is 0.35 and jumps to on average 0.41.
- However, we concluded the test with 8241 and it improves only 500 of client on average. 
- This is a huge increase if we run on real time. For instance, if we have  10 milions vistors in each days, it would be 600k of increase in each days of vistores of logins from 3.5 million to 4.1 milion


## Basket CTR
- Variation Screen has an impact to clients of basket creating almost 60% on average. It is 0.12 and jumps to on average 0.18.
- However, we concluded the test with 8241 and it improves only 500 of client on average. 
- This is a huge increase if we run on real time. For instance, if we have  10 milions visitors in each days, it would be 600k of increase in each days of visitors of adding any item to basket count from 1.2 million to 1.8 milion


# Payment Screen CTR
- Variation Screen has an impact to clients of order screen clicks almost 38 % on average It is 0.05 and jumps to on average 0.08.
- However, we concluded the test with 8241 and it improves only 247 of client on average. 
- This is a huge increase if we run on real time. For instance, if we have 10 milions visitors in each days, it would be 300k of increase in each days of visitores of payment screen click count from 500k to 800k milion.
### It seems that when we continue our test enviroment this CTR values will close to each other more. That is why wee need to carry on data collecting for payment screen.

# Order CTR
- Variation Screen has an impact to clients of basket creating count almost 38% on average It is 0.05 and jumps to on average 0.08.
- However, we concluded the test with 8241 and it improves only 247 of client on average. 
- This is a huge increase if we run on real time. For instance, if we have 10 milions visitors in each days, it would be 300k of increase in each days of visitores of payment screen click count from 500k to 800k milion.
### This CTR values are also not well tested too. It must be more data in order to get the actual CTR improvements. Test must be continued for Order count CTR.

In [ ]:
visualization = daily_ab_test_results.pivot_table(index=['day', 'metrics'] , aggfunc={'p_control': 'mean', 
                                                                     'p_validation': 'mean'}).reset_index()

for metric in list(visualization['metrics'].unique()):
    print("metric :", metric)
    _df = visualization.query("metrics == @metric")
    x = [str(i)[0:10] for i in _df['day']]
    y = _df['p_control'].values
    plt.plot(x, y)
    y = _df['p_validation'].values
    plt.plot(x, y)
    plt.xticks(x, x, rotation='vertical')
    plt.show()

In [ ]:
date = list(visualization.sort_values(by = 'day', ascending=False)['day'])[5]
visualization['avg_diff_of_CTR'] = visualization['p_validation'] - visualization['p_control']
visualization_1 = visualization.query("day >= @date").pivot_table(index='metrics', 
                                                                  aggfunc={'avg_diff_of_CTR': 'mean'}).reset_index()
visualization_1['client_count'] = 10000000 * visualization_1['avg_diff_of_CTR']




In [ ]:
objects = list(visualization_1.metrics)
y_pos = np.arange(len(objects))
performance_1 = list(visualization_1.avg_diff_of_CTR)
plt.bar(y_pos, performance_1, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('CTR')
plt.title('Metrics of CTR Differences From Control to Validation')
plt.show()
objects = list(visualization_1.metrics)
y_pos = np.arange(len(objects))
performance_2 = list(visualization_1.client_count)
plt.bar(y_pos, performance_2, align='center', alpha=0.5)
plt.ylabel('client count difference After New Screen seen')
plt.title('What If This Company Has 10 millions of Visitors Daily')
plt.show()

# Segments of CTR Values

In [ ]:
visualization_2 = daily_ab_test_results_rfm.pivot_table(index=['day', 'metrics', 'rfm'] , aggfunc={'p_control': 'mean', 
                                                                                                    'p_validation': 'mean'}).reset_index()

for rfm in list(visualization_2['rfm'].unique()): 
    print(" segment :", rfm)
    for metric in list(visualization_2['metrics'].unique()):
        _df = visualization_2.query("metrics == @metric and rfm == @rfm")
        x = [str(i)[0:10] for i in _df['day']]
        y = _df['p_control'].values
        plt.plot(x, y, label= metric.split('_')[0] + ' c.' )
        y = _df['p_validation'].values
        plt.plot(x, y, label= metric.split('_')[0] + ' v.' )
        plt.grid(True)
        plt.xticks(x, x, rotation='vertical')
    plt.legend()
    plt.show()
    



### A/B Test Segment Explation:
- Segmentaion is created with the popular markettn client segmentation which is RFM (Recency - Frequency- Monetary)
- Recency: How frequently client is ordered.
- Frequency: How frequently client visits.
- Monetary: represents average of basket values of clients.
- Each metrics segmented into 4 groups. There are at most 4 *4 * = 64 group can be created. 1; represents the best, 4; the worst segment.
- Some segments need more data as like 3_1_3.
- 1_1_3, 2_2_2, 3_2_1, 3_2_2, 3_3_1, 3_3_2, 3_4_1 segmets are well has better impact pn validation sets.

In [ ]:
best_segments = ['1_1_3', '2_2_2', '3_2_1', '3_2_2', '3_3_1', '3_3_2', '3_4_1']
fequencies = pd.DataFrame([int(rfm.split('_')[0]) for rfm in best_segments]).rename(columns={0: 'frequency'})
fequencies['freq_count'] = fequencies['frequency']
fequencies = fequencies.pivot_table(index='frequency', aggfunc={'freq_count': 'count'}).reset_index()
recencies = pd.DataFrame([int(rfm.split('_')[1]) for rfm in best_segments]).rename(columns={0: 'recency'})
recencies['r_count'] = recencies['recency']
recencies = recencies.pivot_table(index='recency', aggfunc={'r_count': 'count'}).reset_index()
monetaries = pd.DataFrame([int(rfm.split('_')[2]) for rfm in best_segments]).rename(columns={0: 'monetary'})
monetaries['m_count'] = monetaries['monetary']
monetaries = monetaries.pivot_table(index='monetary', aggfunc={'m_count': 'count'}).reset_index()

In [ ]:
display(fequencies, recencies, monetaries)

### Results of segmented clients:
- The client who pays much loves the new screen :)
- However, these clients are not frequently vistors.

# For Next Analysis
- This segmented clients of attributtes must be investiged. 
- Are their frequency segments are changing?  
- What is their order period has change?

for more information pls check gitHub reporsitory:
https://github.com/caglanakpinar/a_b_test

# Dashboard to Visualization Outputs

- I use dash library to visualize outputs.
- Make sure you install dash library. Both conda install and pip install coments are worked on that.
- In addition to Dash library, you need to install plotly too.
- In order to use plotly library you need license. However, they allow us to use it locally offline 

In [ ]:
# check these libraries can be imported successfully
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()

- run the code .ipynb notebook below.
- Make sure is at same directory as current file.
- Make sure 'daily_ab_test_results.csv' and 'daily_ab_test_results_rfm.csv' are in the same directory.
- If there aren`t any of these .csv file, you can use defaults

In [ ]:
%run ./visualization.ipynb